In [1]:
!pwd

/home/campbell/yulee/DAMM/new/code/dev


In [2]:
import sys
sys.path.append('/home/campbell/yulee/DAMM/new/code/pro')

from damm_funcs import *

In [3]:
cohort = 'basel'
ss = 1
nc = 3
analysisProteins = ['CD20', 'CD3', 'CD44', 'CD45', 'CD68', 'Carbonic_Anhydrase_IX',
       'Cytokeratin19', 'Cytokeratin5', 'Cytokeratin7', 'Cytokeratin8/18',
       'DNA1', 'DNA2', 'EGFR', 'Ecadherin', 'EpCAM', 'Erk1/2', 'Fibronectin',
       'GATA3', 'H3K27me3', 'Keratin14', 'Ki67', 'Progesterone_Receptor_A/B',
       'RabbitIgGHL', 'S6', 'SMA', 'Slug', 'Sox9', 'TotalHH3', 'Twist', 
       'Vimentin', 'bCatenin', 'cMyc', 'cerbB2_Her2', 'cleavedPARP_CleavedCaspase3',
       'mTOR', 'p53', 'panCytokeratin/Keratin_Epithelial', 'phospho_HH3', 'vWF_CD31']

tr_exp = sc.read_h5ad('/home/campbell/yulee/DAMM/new/data/{}/{}k/tr_exp.h5ad'.format(cohort, ss))
#'DAMM/new/data/{cohort}/{ss}k/te_exp.h5ad'

Observation names are not unique. To make them unique, call `.obs_names_make_unique`.


In [4]:
trY, trS, Theta = prepData(tr_exp, nc, analysisProteins)

In [5]:
psi = torch.exp(Theta['log_psi'])
omega = torch.exp(Theta['log_omega'])

psi2 = psi.reshape(-1,1)
psi2 = psi2 + psi2.T

omega2 = omega.reshape(-1,1)
omega2 = omega2 + omega2.T + 1e-6

In [47]:
#import itertools
#for a, b in itertools.combinations(psi, r=2):
#    print(max(a, b))

tensor(109.2520, dtype=torch.float64, grad_fn=<UnbindBackward>)
tensor(113.9273, dtype=torch.float64, grad_fn=<UnbindBackward>)
tensor(113.9273, dtype=torch.float64, grad_fn=<UnbindBackward>)


In [6]:
ccmax = torch.combinations(psi).max(1).values
mat = torch.zeros(nc, nc, dtype=torch.float64)

In [7]:
mat[np.triu_indices(nc, 1)] = ccmax
mat += mat.T
mat += torch.eye(nc) * psi

In [8]:
psi

tensor([ 98.0721, 109.2520, 113.9273], dtype=torch.float64,
       grad_fn=<ExpBackward>)

In [9]:
mat

tensor([[ 98.0721, 109.2520, 113.9273],
        [109.2520, 109.2520, 113.9273],
        [113.9273, 113.9273, 113.9273]], dtype=torch.float64,
       grad_fn=<AddBackward0>)

In [10]:
psi2

tensor([[196.1442, 207.3241, 211.9994],
        [207.3241, 218.5039, 223.1792],
        [211.9994, 223.1792, 227.8545]], dtype=torch.float64,
       grad_fn=<AddBackward0>)

In [130]:
#mat = np.zeros((nc, nc), float)
#np.fill_diagonal(mat, psi.detach().numpy())
#mat[np.triu_indices(nc, 1)] = torch.combinations(psi).max(1).values.detach().numpy()
#mat[np.tril_indices(nc, -1)] = torch.combinations(psi).max(1).values.detach().numpy()

In [128]:
#D.Uniform(torch.tensor(mat, requires_grad=True), psi2)

Uniform(low: torch.Size([3, 3]), high: torch.Size([3, 3]))

In [11]:
psi2 > mat

tensor([[True, True, True],
        [True, True, True],
        [True, True, True]])

In [12]:
## v ~ Uniform(max(mu_c, mu_c’), mu_c + mu_c’)
## p(s|d=1, gamma=[c,c’], v) = ½ [erf(q * (c ** 0.5)) + erf(p * (c ** 0.5))]

#c = (1 / (2 * (omega2 ** 2)))
c = 1 / (np.sqrt(2) * omega2)
p = mat - trS.reshape(-1,1,1)
q = psi2 - trS.reshape(-1,1,1)

In [13]:
c.shape

torch.Size([3, 3])

In [14]:
p.shape

torch.Size([1000, 3, 3])

In [15]:
q.shape

torch.Size([1000, 3, 3])

In [16]:
#ubp = torch.special.erf(q * torch.sqrt(c))
#lbp = torch.special.erf(p * torch.sqrt(c))

const = 1/(2 * (psi2 - mat))
lbp = torch.special.erf(p * c)
ubp = torch.special.erf(q * c)
prob = const * (ubp - lbp)

In [17]:
prob.log().shape

torch.Size([1000, 3, 3])

In [18]:
prob

tensor([[[0.0027, 0.0025, 0.0024],
         [0.0025, 0.0024, 0.0023],
         [0.0024, 0.0023, 0.0023]],

        [[0.0036, 0.0036, 0.0035],
         [0.0036, 0.0036, 0.0035],
         [0.0035, 0.0035, 0.0034]],

        [[0.0019, 0.0021, 0.0022],
         [0.0021, 0.0022, 0.0023],
         [0.0022, 0.0023, 0.0023]],

        ...,

        [[0.0035, 0.0035, 0.0034],
         [0.0035, 0.0035, 0.0034],
         [0.0034, 0.0034, 0.0033]],

        [[0.0026, 0.0024, 0.0023],
         [0.0024, 0.0023, 0.0022],
         [0.0023, 0.0022, 0.0022]],

        [[0.0023, 0.0021, 0.0020],
         [0.0021, 0.0019, 0.0019],
         [0.0020, 0.0019, 0.0018]]], dtype=torch.float64,
       grad_fn=<MulBackward0>)

In [19]:
sum(prob > 1)

tensor([[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]])

In [20]:
sum(prob <= 0)

tensor([[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]])

In [21]:
#dist_S2 = D.Normal(loc = psi2, scale = omega2)
dist_S2 = D.StudentT(df = psi2, loc = psi2, scale = omega2)

In [22]:
dist_S2.log_prob(trS.reshape(-1, 1, 1)).exp()

tensor([[[0.0018, 0.0015, 0.0015],
         [0.0015, 0.0013, 0.0012],
         [0.0015, 0.0012, 0.0012]],

        [[0.0034, 0.0033, 0.0032],
         [0.0033, 0.0031, 0.0030],
         [0.0032, 0.0030, 0.0029]],

        [[0.0028, 0.0031, 0.0031],
         [0.0031, 0.0033, 0.0033],
         [0.0031, 0.0033, 0.0033]],

        ...,

        [[0.0031, 0.0029, 0.0028],
         [0.0029, 0.0027, 0.0025],
         [0.0028, 0.0025, 0.0024]],

        [[0.0017, 0.0014, 0.0014],
         [0.0014, 0.0012, 0.0012],
         [0.0014, 0.0012, 0.0011]],

        [[0.0013, 0.0011, 0.0011],
         [0.0011, 0.0009, 0.0009],
         [0.0011, 0.0009, 0.0009]]], dtype=torch.float64,
       grad_fn=<ExpBackward>)

In [23]:
def compute_p_s_given_gamma2(S, Theta, modelType):
  
    """ NxCxC
    p(y_n | gamma_n = [c,c'])
    """

    psi = torch.exp(Theta['log_psi'])
    omega = torch.exp(Theta['log_omega'])

    psi2 = psi.reshape(-1,1)
    psi2 = psi2 + psi2.T

    omega2 = omega.reshape(-1,1)
    omega2 = omega2 + omega2.T + 1e-6

    if modelType == 'normal':
        dist_S2 = D.Normal(loc = psi2, scale = omega2)
    elif modelType == 'student':
        dist_S2 = D.StudentT(df = 2.0, loc = psi2, scale = omega2)
    else:
        ## for v
        ccmax = torch.combinations(psi).max(1).values
        mat = torch.zeros(len(psi), len(psi), dtype=torch.float64)    
        mat[np.triu_indices(len(psi), 1)] = ccmax
        mat += mat.T
        mat += torch.eye(len(psi)) * psi
        #v = D.Uniform(mat, psi2).sample()
        
        ## for s
        #c = (1 / (2 * (omega2 ** 2)))
        c = 1 / (np.sqrt(2) * omega2)
        q = psi2 - trS.reshape(-1,1,1)
        p = mat - trS.reshape(-1,1,1)
        
        const = 1/(2 * (psi2 - mat))
        #ubp = torch.special.erf(q * torch.sqrt(c))
        #lbp = torch.special.erf(p * torch.sqrt(c))
        lbp = torch.special.erf(p * c)
        ubp = torch.special.erf(q * c)
        prob = const * (ubp - lbp)

        return prob.log()
    
    return dist_S2.log_prob(S.reshape(-1, 1, 1))

In [24]:
trY, trS, Theta = prepData(tr_exp, nc, analysisProteins)

In [25]:
s_t1 = compute_p_s_given_gamma2(trS, Theta, 'normal')

In [26]:
s_t2 = compute_p_s_given_gamma2(trS, Theta, 'student')

In [27]:
s_t3 = compute_p_s_given_gamma2(trS, Theta, 'relaxRule')

In [28]:
s_t1.exp()[0]

tensor([[0.0012, 0.0015, 0.0012],
        [0.0015, 0.0018, 0.0015],
        [0.0012, 0.0015, 0.0013]], dtype=torch.float64,
       grad_fn=<SelectBackward>)

In [29]:
s_t2.exp()[0]

tensor([[0.0011, 0.0012, 0.0011],
        [0.0012, 0.0014, 0.0013],
        [0.0011, 0.0013, 0.0011]], dtype=torch.float64,
       grad_fn=<SelectBackward>)

In [30]:
s_t3.exp()[0]

tensor([[0.0023, 0.0024, 0.0023],
        [0.0024, 0.0027, 0.0025],
        [0.0023, 0.0025, 0.0024]], dtype=torch.float64,
       grad_fn=<SelectBackward>)

In [4]:
tr_file = 'DAMM/new/data/{}/{}k/tr_exp.h5ad'.format(cohort, ss)
te_file = 'DAMM/new/data/{}/{}k/te_exp.h5ad'.format(cohort, ss)

tr_exp = sc.read_h5ad(tr_file)
te_exp = sc.read_h5ad(te_file)

Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
Observation names are not unique. To make them unique, call `.obs_names_make_unique`.


In [3]:
BatchSize = 128
nm = 'student'
learnRate = 1e-3
so = None
tr_so = None
rr = 0
trY, trS, Theta = prepData(tr_exp, nc, analysisProteins)
loss, Theta = trainWOFD(trY, trS, Theta, BatchSize, nm, learnRate, so, tr_so, rr)

KeyboardInterrupt: 

In [5]:
#python3 hello.py --train DAMM/new/data/basel/1k/tr_exp.h5ad 
#--test DAMM/new/data/basel/1k/te_exp.h5ad --cohort basel --sample_size 1 --n_cluster 9 
#--noise_model normal --regVal 1000000 --repetition 0 
#--output_theta0 DAMM/new/res/basel/1k/damm_theta0_nc9_normal_rv1000000_soNone_rr1_0 
#--output_theta1 DAMM/new/res/basel/1k/damm_theta1_nc9_normal_rv1000000_soNone_rr1_0 
#--so None --rr 1

nc = 9
BatchSize = 128
nm = 'normal'
learnRate = 1e-3
so = None
tr_so = None
rr = 1
rv = 1000000

trY, trS, Theta = prepData(tr_exp, nc, analysisProteins)
trFY, trFS, trFL = simulate_data(trY, trS)

teY, teS, _ = prepData(te_exp, nc, analysisProteins)
teFY, teFS, teFL = simulate_data(teY, teS)

df = ConcatDataset(trY, trS, trFY, trFS, trFL)
loss, Theta = trainWFD(df, Theta, BatchSize, nm, learnRate, rv, so, tr_so, rr)

In [39]:
compute_p_s_given_gamma2(trS, Theta, nm, 1)

1


tensor([[[-5.6394, -5.8188, -6.2714,  ..., -5.7537, -6.0562, -5.9312],
         [-5.8458, -6.0985, -6.3454,  ..., -6.1328, -5.9035, -6.4123],
         [-5.8444, -5.9287, -6.4665,  ..., -5.9105, -5.8162, -5.9056],
         ...,
         [-6.4045, -5.9222, -6.2553,  ..., -6.1323, -5.9205, -6.2536],
         [-5.6517, -5.8541, -5.7912,  ..., -6.0213, -5.7401, -6.1722],
         [-5.7406, -5.9466, -6.3327,  ..., -5.7927, -6.2810, -5.8521]],

        [[-5.5684, -5.5923, -5.6981,  ..., -5.5191, -5.5804, -5.6551],
         [-5.5876, -5.6732, -5.7913,  ..., -5.6081, -5.6494, -5.8287],
         [-5.6507, -5.7154, -5.8856,  ..., -5.6433, -5.7355, -5.7923],
         ...,
         [-5.5996, -5.5842, -5.6950,  ..., -5.5282, -5.5718, -5.7084],
         [-5.6648, -5.6566, -5.7514,  ..., -5.5777, -5.6904, -5.7418],
         [-5.7187, -5.7278, -5.8456,  ..., -5.6875, -5.7723, -5.8408]],

        [[-6.8170, -6.4188, -5.9187,  ..., -6.5352, -6.0899, -6.2589],
         [-6.3702, -6.0617, -5.9014,  ..., -6